In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import time

Using TensorFlow backend.
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [3]:
env = gym.make('CartPole-v1')
print(env.observation_space.shape)

(4,)


In [4]:
if __name__ == "__main__":
    start_time = time.time()
    EPISODES = 100
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32
    
    for e in range(EPISODES):
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for t in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, t, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")
    elapsed_time = time.time() - start_time
    print(elapsed_time)

episode: 0/100, score: 57, e: 0.88
episode: 1/100, score: 18, e: 0.81
episode: 2/100, score: 18, e: 0.74
episode: 3/100, score: 13, e: 0.69
episode: 4/100, score: 8, e: 0.66
episode: 5/100, score: 25, e: 0.58
episode: 6/100, score: 13, e: 0.55
episode: 7/100, score: 11, e: 0.52
episode: 8/100, score: 8, e: 0.5
episode: 9/100, score: 12, e: 0.47
episode: 10/100, score: 10, e: 0.45
episode: 11/100, score: 12, e: 0.42
episode: 12/100, score: 11, e: 0.4
episode: 13/100, score: 7, e: 0.38
episode: 14/100, score: 38, e: 0.32
episode: 15/100, score: 19, e: 0.29
episode: 16/100, score: 12, e: 0.27
episode: 17/100, score: 72, e: 0.19
episode: 18/100, score: 96, e: 0.12
episode: 19/100, score: 44, e: 0.094
episode: 20/100, score: 106, e: 0.055
episode: 21/100, score: 79, e: 0.037
episode: 22/100, score: 63, e: 0.027
episode: 23/100, score: 51, e: 0.021
episode: 24/100, score: 85, e: 0.014
episode: 25/100, score: 55, e: 0.01
episode: 26/100, score: 55, e: 0.01
episode: 27/100, score: 87, e: 0.01
